In [9]:
import os
import tarfile
import gzip
import numpy as np
import csv
import shutil
import subprocess
import mne
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.use('TkAgg')

In [8]:
# def extract_and_convert(data_dir):
#     Extract .tar.gz files
#     for file in os.listdir(data_dir):
#         if file.endswith(".tar.gz"):
#             file_path = os.path.join(data_dir, file)
#             with tarfile.open(file_path, 'r:gz') as tar:
#                 tar.extractall(path=data_dir)
#                 print(f"Extracted {file} successfully.")
    
#     Extract .gz files
#     print("Extracting...")
#     for file in os.listdir(data_dir):
#         file_path = os.path.join(data_dir, file)
#         if not os.path.isdir(file_path):
#             continue
#         if file == '.DS_Store' or file == 'README':
#             continue
#         for filename in os.listdir(file_path):
#             fname = os.path.join(file_path, filename)
#             if fname.endswith(".gz"):
#                 subprocess.call(['gunzip', '-q', fname])
#                 print(f"Extracted {fname} successfully.")

    # for root, dirs, files in os.walk(data_dir):
    #     for dir_name in dirs:
    #         dir_path = os.path.join(root, dir_name)
    #         print("Directory:", dir_path)
        
    #         # Iterate through files in the current directory
    #         for file in os.listdir(dir_path):
    #             file_path = os.path.join(data_dir, file)
    #             if not os.path.isdir(file_path):
    #                 continue
    #             if file == '.DS_Store' or file == 'README':
    #                 continue
    #             for filename in os.listdir(file_path):
    #                 fname = os.path.join(file_path, filename)
    #                 if fname.endswith(".gz"):
    #                     subprocess.call(['gunzip', '-q', fname])
    #                     print(f"Extracted {fname} successfully.")

            # Convert to CSV
#             for root, _, files in os.walk(data_dir):
#                 for file in files:
#                     if '.rd.' in file and not file.endswith('.gz'):

#                         subject = os.path.basename(root)
#                         subject_trial = file
#                         convert_to_csv(subject, subject_trial)

# def convert_to_csv(subject, subject_trial):
#     file_path = os.path.join(data_dir, subject, subject_trial)
#     with open(file_path, 'r') as f:
#         lines = f.readlines()

#     data = []
#     for line in lines:
#         if not line.startswith('#'):
#             parts = line.strip().split()
#             data.append(parts)

#     csv_file_name = os.path.join(data_dir, subject, f'{subject_trial}.csv')

#     with open(csv_file_name, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Trial Number', 'Sensor Position', 'Sample Number', 'Sensor Value (µV)'])
#         writer.writerows(data)

#     print(f"Data has been saved to {csv_file_name}.")

# # Example usage:
# data_dir = '/Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/'
# extract_and_convert(data_dir)

Extracting...
Data has been saved to /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.009.csv.
Data has been saved to /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.031.csv.
Data has been saved to /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.007.csv.
Data has been saved to /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.000.csv.
Data has been saved to /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.001.csv.
Data has been saved to /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.006.csv.
Data has been saved to /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.037.csv.
Data has been saved to /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.008.csv.
Data has been saved to /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co

KeyboardInterrupt: 

122 subjects
120 trials
1 sec per trial


In [17]:
# Define the channels
channels = ["FP1", "AF7", "AF8", "FC4", "FC5", "FC6", "T7", "CZ", "PO7", "PO8", "O1", "OZ", "FP2", "O2", "T8", "FC3"]

# Initialize data and label arrays
data_array = {}
label_array = {}

data_dir = '/Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/'

import os
import csv
import numpy as np

data_array = {}  # Dictionary to store data arrays

# Define epoch length (in samples)
epoch_length = 256  # Assuming 1 second corresponds to 256 samples

for root, dirs, files in os.walk(data_dir):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        print("Directory:", dir_path)
        
        label = dir_name[3] #co2a0000365 the fourth letter (here it is 'a') informs if it is an alcoholic or a control person
        label_value = 0
        if label == 'c': # Non-Alcohol
            label_value = 0
        elif label == 'a':  # Alcohol
            label_value = 1

        all_trial_data = []
        # Iterate through files in the current directory
        for file_name in os.listdir(dir_path):
            if file_name.endswith('.csv'):
                file_path = os.path.join(dir_path, file_name)
                print("File:", file_path)
                
                # Example: Open the CSV file and read its contents
                trial_data = []
                print("Read csv: ")
                with open(file_path, mode='r') as file:
                    reader = csv.DictReader(file)
                    for row in reader:
                        sensor_position = row['Sensor Position']
                        if sensor_position in channels:
                            # print(sensor_position)
                            sensor_value = float(row['Sensor Value (µV)'])  # Assuming sensor value is numeric
                            trial_data.append(sensor_value)  # Extract sensor values from the fourth column
                # Accumulate trial data
                all_trial_data.append(trial_data)
        
        # Organize trial data into epochs
        num_trials = len(all_trial_data)
        print("Number of trials:", num_trials)
        if num_trials > 0:
            data_key = f"{dir_name}_X"
            label_key = data_key.replace('_X', '_Y')
            data_array[data_key] = np.zeros(shape=(num_trials, len(channels), epoch_length))
            label_array[label_key] = np.full(shape=(num_trials,), fill_value=label_value)
            for i in range(num_trials):
                trial_data = all_trial_data[i]
                for j in range(len(channels)):
                    channel_data = trial_data[j * epoch_length: (j + 1) * epoch_length]
                    data_array[data_key][i, j, :] = channel_data
print(data_array)
print(label_array)

print("Data and label arrays created successfully.")
print(data_array)
print(label_array)


Directory: /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372
File: /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.016.csv
Read csv: 
File: /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.002.csv
Read csv: 
File: /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.003.csv
Read csv: 
File: /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.017.csv
Read csv: 
File: /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.001.csv
Read csv: 
File: /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.015.csv
Read csv: 
File: /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.029.csv
Read csv: 
File: /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.rd.014.csv
Read csv: 
File: /Users/sunniva/Downloads/eeg+database/SMNI_CMI_TRAIN/co2a0000372/co2a0000372.r

In [18]:
from scipy.signal import resample

# Define the new number of samples to match the model's training data
new_samples = 250  # New number of samples to match the model's training data

# Resample each trial in data_array to match the new number of samples
resampled_data_array = {}
for key, data in data_array.items():
    num_trials, num_channels, original_samples = data.shape
    resampled_data = np.zeros((num_trials, num_channels, new_samples))
    for i in range(num_trials):
        for j in range(num_channels):
            resampled_data[i, j, :] = resample(data[i, j, :], new_samples)
    resampled_data_array[key] = resampled_data

    # Print the shape of the resampled data
    print(f"Resampled data shape for {key}: {resampled_data.shape}")

    # Print the resampled data for the first trial and channel
    print(f"Resampled data for the first trial and channel for {key}:")
    print(resampled_data[0, 0, :])

Resampled data shape for co2a0000372_X: (30, 16, 250)
Resampled data for the first trial and channel for co2a0000372_X:
[ 2.41534009  1.91689116 -0.17586517 -2.03317169 -2.50034279 -1.80942631
 -0.36879685 -0.09605668 -1.26812592 -2.07571013 -2.63477499 -2.96401104
 -3.04899247 -3.62185022 -4.09671292 -4.52719857 -4.22867903 -3.8620999
 -4.10213061 -4.69768616 -4.97363823 -4.61129449 -3.00874809 -2.53882839
 -3.66699172 -6.0914181  -8.81623539 -9.78415685 -8.7882984  -5.91575101
 -4.91714604 -5.4943909  -7.08832128 -8.13255107 -8.51031535 -6.74621008
 -4.27093434 -2.15551283 -0.63306726 -0.44844692 -1.02903044 -1.44724117
 -2.02674076 -2.44535833 -2.49334891 -2.47467626 -3.00865679 -2.96171156
 -3.50640161 -2.767617   -2.5126181  -2.44417271 -3.34897607 -5.62258156
 -8.67072504 -9.93847056 -9.41713027 -6.66828452 -4.09559968 -2.30684816
 -2.27521411 -3.58957883 -5.23764088 -6.22438126 -6.4535271  -6.12470008
 -5.45648408 -3.77445206 -1.52930893  1.48984027  3.58157694  4.05914775
  2.6

In [19]:
# Specify the file path
data_file_path = "resampled_data.npy"

# Save the resampled data array to a NumPy file
np.save(data_file_path, resampled_data_array, )

In [20]:
# Specify the file path
label_file_path = "labels.npy"

# Save the label array to a NumPy file
np.save(label_file_path, label_array)

: 